In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os
from sqlalchemy import create_engine
import json

In [2]:
# Create folder and popule with enem data
data_path = './enem2019'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    # Define url
    url = 'http://download.inep.gov.br/microdados/microdados_enem_2019.zip'

    # Content download
    filebytes = BytesIO(
        requests.get(url, stream=True).content
    )

    # Exctract content from zipfile
    myzip = zipfile.ZipFile(filebytes)
    myzip.extractall(data_path)

In [3]:
csv_file = './enem2019/DADOS/MICRODADOS_ENEM_2019.csv'

In [4]:
# Create functin that splits csv into multiple csvs
# Source code: https://gist.github.com/jrivero/1085501

def split(filehandler, delimiter=',', row_limit=1000,
          output_name_template='output_%s.csv', output_path='.', keep_headers=True):
    import csv
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)

In [5]:
# Split csv into csv files of 'row_number' rows, so creating pandas dataframe is possible.
# If there's not much RAM available, consider putting the 'row_number' variable as low as possible.
# You can get good result splitting the CSV into files of, for example, 20.000 rows.

# In my local computer, splitting it in files of 1 million rows each was good enough, but more than that starts to 
# overflow my memory when creating pandas dataframe
row_number = 1000000

split(open(csv_file, encoding='latin-1'), delimiter=';', row_limit=row_number,
     output_name_template='microdados_enem_%s.csv', output_path=data_path+'/DADOS/splitted_files', keep_headers=True)

In [6]:
# Read each csv into a pandas dataframe, look for SG_UF_RESIDENCIA = 'MG' and replace each csv only with MG data.
directory = data_path+'/DADOS/splitted_files/'

for filename in sorted(os.listdir(directory)):
    df = pd.read_csv(directory+filename, sep = ';', decimal = '.', encoding='latin-1')
    df = df.loc[
    df.SG_UF_RESIDENCIA == 'MG'
    ]
    os.remove(directory+filename)
    df.to_csv(directory+filename, index=False, sep = ';', decimal = '.', encoding='latin-1')

In [7]:
# Get database access variables defined in vaccess_pg
with open('vaccess_pg.txt') as access_file:
    vaccess = json.load(access_file)

In [8]:
# Connect to database and bulk insert each csv into the table
# Adapted from https://www.dataquest.io/blog/loading-data-into-postgres/
import psycopg2
conn = psycopg2.connect(host=vaccess['host'], dbname=vaccess['dbname'], user=vaccess['user'], password=vaccess['password'])
cur = conn.cursor()


directory = data_path+'/DADOS/splitted_files/'

try:
    for filename in sorted(os.listdir(directory)):
        print(filename)
        if filename.endswith(".csv"):
            enem = pd.read_csv(
                    directory+filename, sep = ';', decimal = '.', encoding='latin-1'
            )

            with open(directory+filename, 'r') as f:
                next(f) 
                cur.copy_from(f, 'enem', sep=';', null="")

            conn.commit()

        else:
            continue
finally:
    cur.close()
    conn.close()

microdados_enem_1.csv
microdados_enem_2.csv
microdados_enem_3.csv
microdados_enem_4.csv
microdados_enem_5.csv
microdados_enem_6.csv
